## Introduction

This notebook will introduce the concept of selecting sets of rows and columns in each of pandas, an SQL table, and a NoSQL collecion.

Along the way, you will learn some methods for interacting with SQL and NoSQL in MySQL.

## What you already know

Using DataFrames, you're already well-versed in filtering your data to produce a subset of columns or rows.

For example, you can work with the provided csv, Calgary_Public_Library_Locations_and_Hours.csv.

Open it now using pandas.

In [1]:
import pandas as pd

cpl_locations = pd.read_csv("Calgary_Public_Library_Locations_and_Hours.csv")
cpl_locations.head()

,Library,Postal_Code,Square_Feet,Phone_Number,Monday_Open,Monday_Close,Tuesday_Open,Tuesday_Close,Wednesday_Open,Wednesday_Close,Thursday_Open,Thursday_Close,Friday_Open,Friday_Close,Saturday_Open,Saturday_Close,Sunday_Open,Sunday_Close,Address
0,W.R. Castell Central Library,T2G 2M2,177532,403-260-2600,9:00,20:00,9:00,20:00,9:00,20:00,9:00,20:00,9:00,17:00,10:00,17:00,12:00,17:00,"616 Macleod Tr SE\n(51.0470276, -114.0578995)"
1,Alexander Calhoun Library,T2T 3V8,9256,403-260-2600,10:00,21:00,10:00,21:00,10:00,21:00,10:00,21:00,10:00,18:00,10:00,17:00,12:00,17:00,"3223 14 St SW\n(51.0255318, -114.0947876)"
2,Bowness Library,T3B 0H3,7800,403-260-2600,13:00,20:00,10:00,20:00,10:00,20:00,10:00,17:00,10:00,17:00,10:00,17:00,12:00,17:00,"7930 Bowness Rd NW\n(51.0872841, -114.1830978)"
3,Fish Creek Library,T2J 6S1,47074,403-260-2600,9:00,21:00,9:00,21:00,9:00,21:00,9:00,21:00,9:00,18:00,9:00,17:00,12:00,17:00,"11161 Bonaventure Dr SE\n(50.9516296, -114.060..."
4,Forest Lawn Library,T2A 4M1,16977,403-260-2600,10:00,21:00,10:00,21:00,10:00,21:00,10:00,21:00,10:00,18:00,10:00,17:00,12:00,17:00,"4807 8 Av SE\n(51.045105, -113.9652023)"


Now for some practice filtering subsets of rows (projections) and columns (selections).

In the code block below, write code for:
- the subset of rows for all libraries which are open at 9:00 AM on Fridays. 
- the subset of rows for all libraries under 8000 square feet in area
- the subset of columns consisting of the library name, the address, and the postal code

In [36]:
# your pandas code should go here

## Getting set up to use mySQL

To connect to your mysql database, you will need a *connector*. There are many of these available, but we will be using [`mysql-connector-python,`](https://anaconda.org/anaconda/mysql-connector-python]) as one of the most common and well-documented ways to connect to MySql. You may need to install the connector first. 

Use the following command in your Jupyter terminal:

`pip install mysql-connector-python` 
or
`conda install mysql-connector-python`

(If you have followed our previous instructions on how to set up mySQL, this should already be done)




## <a name = "createdb"></a> Create and populate a database table

You should have a database named `test` as part of your setup (we had you create it). We will use this for now.

Now we'll create a table using the MySQL shell. We can do this using Python, but we want you to have some familiarity with the MySQL shell as well.

Use the file named `create_libraries.sql` to do so. Be sure to open it in a text editor beforehand to check what it does. 

To use `create_libraries.sql`, from the MySQL shell, do the following:
- use sql mode (`\sql`)
- connect to your database if you aren't already connected (`\connect root@localhost:33060`)
- select your schema (`\use test`)
- use the command (`\. "<PATH TO FILE>\create_libraries.sql";` or `\source "PATH TO FILE\create_libraries.sql";\`) to get MySQL to read the contents of the file and run it
- you may use `show tables;` and `describe library_locations;` to verify that everything has run correctly

Now we are going to insert some data into the table. We start by connecting to the database.


In [2]:
import mysql.connector

# fill in any relevant details that need to be changed here, such as if you set up a different user or password
myconnection = mysql.connector.connect(user='root', password='Leftw!ng6977',
                                 host='127.0.0.1',
                                 database='test')

The next step is to load data into the table. There are again many approaches, but we note that the data has already been loaded into a DataFrame. 

The first thing we need is something to hold our place in the table, commonly called a cursor.




In [3]:
insertCursor = myconnection.cursor()

columnString = "`,`".join([str(currentColumn) for currentColumn in cpl_locations.columns.tolist()])
#print (columnString)

# inserting rows one by one from the DataFrame is sufficient for now
for i, currentRow in cpl_locations.iterrows():
    #print (tuple(currentRow))
    insertCommand = "INSERT INTO `library_locations` (`" + columnString + "`) VALUES (" + "%s,"*(len(currentRow)-1) + "%s)"
    #print (insertCommand)
    insertCursor.execute(insertCommand, tuple(currentRow))
    
myconnection.commit()
myconnection.close()

Should you need to reset your database at any point in time, use the following SQL command to drop the table:
`DROP TABLE library_locations;`

Then [return to the notebook cell](#createdb) about creating a table and follow the notebook to the point where data has been inserted again.

## Querying a database table

The structure of a SQL is relatively simple. 

SELECT {COLUMNS} FROM {TABLE} WHERE {conditions}

For example, to select the branch names and addresses from our library_location table, we would use:

`SELECT Library, Address FROM library_location`

A query which selects a subset of columns from a table is called a _projection_.

A query which selects a subset of rows from a table is called a _selection_.

The following query is a selection of all rows of libraries with the word "Hill" in their library name.

`SELECT * from library_locations where Library LIKE '%hill%';`

You will notice that SQL is not particularly case sensitive.

Now from the MySQL shell, write queries in the shell for the following:
- the subset of columns consisting of the library name, and the phone number
- the subset of rows for all libraries which are open at 12:00 PM on Sundays. (_Hint: you should be able to use = for equality_)
- the subset of rows for all libraries over 12000 square feet in area (_Hint: you should be able to use < and > with numbers_)

Include your queries in the Markdown Cell below for posterity.

SELECT Library FROM library_locations WHERE Sunday_Open = '12:00';
SELECT Library, Sunday_Open FROM library_locations WHERE Sunday_Open = '12:00';
SELECT Library, Square_Feet FROM library_locations WHERE Square_Feet>12000;

## Using NoSQL for queries

The first thing we're going to do is load a DataFrame from a JSON file, and get comfortable with the data.

In [5]:
# JSON: load a DataFrame from JSON

import json

# There are different ways to parse JSON files for DataFrames, but some 
# experimentation may be needed to get your file into the right shape. 

# I encourage you to play around with the lines below to see how this code 
# took shape

with open("canadianCheeseDirectory.json", encoding="UTF-8") as cheeseDirectory:
    inputList = json.load(cheeseDirectory)

# print (inputList)    
    
cheeseFrame = pd.DataFrame.from_dict(inputList)
cheeseFrame = cheeseFrame.CheeseDirectory.dropna().apply(pd.Series)

display(cheeseFrame)




,CheeseId,CheeseNameEn,CheeseNameFr,ManufacturerNameEn,ManufacturerNameFr,ManufacturerProvCode,ManufacturingTypeEn,ManufacturingTypeFr,WebSiteEn,WebSiteFr,...,Organic,CategoryTypeEn,CategoryTypeFr,MilkTypeEn,MilkTypeFr,MilkTreatmentTypeEn,MilkTreatmentTypeFr,RindTypeEn,RindTypeFr,LastUpdateDate
0,228,,Sieur de Duplessis (Le),,Fromages la faim de loup,NB,Farmstead,Fermière,,,...,0,Firm Cheese,Pâte ferme,Ewe,Brebis,Raw Milk,Lait cru,Washed Rind,Croûte lavée,2016-02-03T11:04:24-05:00
1,242,,Tomme Le Champ Doré,,Fromages la faim de loup,NB,Farmstead,Fermière,,,...,0,Semi-soft Cheese,Pâte demi-ferme,Cow,Vache,Raw Milk,Lait cru,Washed Rind,Croûte lavée,2016-02-03T11:04:24-05:00
2,301,Provolone Sette Fette (Tre-Stelle),Provolone Sette Fette (Tre-Stelle),Tre Stelle (Arla Foods),,ON,Industrial,Industrielle,http://www.trestelle.ca/english/,http://www.trestelle.ca/francais/,...,0,Firm Cheese,Pâte ferme,Cow,Vache,Pasteurized,Pasteurisé,,,2016-02-03T11:04:28-05:00
3,303,,Geai Bleu (Le),,Fromages la faim de loup,NB,Farmstead,Fermière,,,...,0,Veined Cheeses,Pâte persillée,Cow,Vache,Raw Milk,Lait cru,,,2016-02-03T11:04:24-05:00
4,319,,Gamin (Le),,Fromages la faim de loup,NB,Farmstead,Fermière,,,...,1,Semi-soft Cheese,Pâte demi-ferme,Cow,Vache,Raw Milk,Lait cru,Washed Rind,Croûte lavée,2016-02-03T11:04:24-05:00
5,350,Paneer (Northumberland Co-operative),,Northumberland Co-operative,,NB,Industrial,Industrielle,http://www.northumberlanddairy.ca/e/0001e.cfm,http://www.northumberlanddairy.ca/f/0001f.cfm,...,0,Fresh Cheese,Pâte fraîche,Cow,Vache,Pasteurized,Pasteurisé,,,2016-02-03T11:04:25-05:00
6,374,Goat Brie (Woolwich),,Woolwich Dairy,,ON,Industrial,Industrielle,http://www.woolwichdairy.com,http://www.woolwichdairy.com/french.aspx,...,0,Soft Cheese,Pâte molle,Goat,Chèvre,Pasteurized,Pasteurisé,Bloomy Rind,Croûte fleurie,2016-02-03T11:04:25-05:00
7,375,Goat Cheddar (Woolwich),,Woolwich Dairy,,ON,Industrial,Industrielle,http://www.woolwichdairy.com,http://www.woolwichdairy.com/french.aspx,...,0,Firm Cheese,Pâte ferme,Goat,Chèvre,Pasteurized,Pasteurisé,,,2016-02-03T11:04:25-05:00
8,376,Goat Mozarella (Woolwich),,Woolwich Dairy,,ON,Industrial,Industrielle,http://www.woolwichdairy.com,http://www.woolwichdairy.com/french.aspx,...,0,Semi-soft Cheese,Pâte demi-ferme,Goat,Chèvre,Pasteurized,Pasteurisé,,,2016-02-03T11:04:25-05:00
9,378,Goat Feta (Woolwich),,Woolwich Dairy,,ON,Industrial,Industrielle,http://www.woolwichdairy.com,http://www.woolwichdairy.com/french.aspx,...,0,Soft Cheese,Pâte molle,Goat,Chèvre,Pasteurized,Pasteurisé,,,2016-02-03T11:04:25-05:00


## Queries to try

- Milk types by English and French
- All cheeses made in the province of Manitoba
- All cheeses which have a website in English or French
- All cheeses made with milk from ewes
- All pasteurized cheeses made in Ontario
- Names of cheese (French and English) produced in Farmsteads


## Loading JSON files into MySQL's document store

Now we're going to use the document store in MySQL. Although MySQL started as a relational database system, this document store gives you the ability to use NoSQL inside MySQL.

This is optional, but we can create a separate schema to separate our two datasets.
To get setup, go into SQL mode in the MySQL shell (if you aren't there already)

`\sql
CREATE DATABASE json_sample;
\use json_sample
\py
`

Then create a Collection

`db.create_collection('cheese')`

To use the document store, we need to use a different set of libraries. Instead of the MySQL Connector, we will be using the MySQL X Protocol, which [is also produced by MySQL](https://dev.mysql.com/doc/x-devapi-userguide). 

The process is similar. Instead of opening a connection, we will open a session. We will use this to access the schema, and then the collection.

We'll begin by adding data from our DataFrame to the collection as a series of documents.



In [180]:
import mysqlx

jsonSession = mysqlx.get_session({'host': 'localhost', 'port':'33060', 'user':'root','password':'yourpasswordhere'})

mySchema = jsonSession.get_schema('json_sample')

myCollection = mySchema.get_collection('cheese')

# first we need to write a document to our collection
# Notice that we have to reshape the DataFrame back into a dictionary
addResult = myCollection.add(cheeseFrame.to_dict(orient="records")).execute()

# check to make sure something actually happened
print(addResult.get_affected_items_count())

1451


## Querying using MySQL shell

The basis of the NoSQL query is the find method.For example, to return everything in the collection (which is called "cheese"), you can type

`db.cheese.find()`

To do a projection, you can use the `fields()` method:

`db.cheese.find().fields(["CheeseId", "CheeseNameEn"])`

To do a selection, you can pass a parameter to the `find()` method:

`db.cheese.find("ManufacturingTypeEn = 'Artisan'")`

You can also use the bind() method to _parameterize_ the query. This allows a query to be submitted without all of the information you want in the filters. 

It is also possible to query the document store programmatically via Python. 

Here is an example:

In [203]:
print(myCollection.count())
queryResult = myCollection.find("ManufacturerProvCode = 'NB'").execute()

rows = queryResult.fetch_all()

for row in rows:
    print("%s %s %s" %(row.CheeseNameEn, row.CheeseNameFr, row.ManufacturerProvCode))


1451
 Sieur de Duplessis (Le) NB
 Tomme Le Champ Doré NB
 Geai Bleu (Le) NB
 Gamin (Le) NB
Paneer (Northumberland Co-operative)  NB
 Cheddar (Blancs d'Arcadie) NB
 Peppercorn (Blancs d'Arcadie) NB
 Tomme Blanche NB
 Tintamarre (Le) NB
 Petit Bayou (Le) NB
 Choupet's (Le) NB
 Saint Manu (Le) NB
 Faisselles (Les) NB
Cheddar  NB
 Fromage en grain NB
 Barbizon (Le) NB
 Cabriolet (Le) NB
 Acadiac (L') NB
 Petit Frais (Le) NB
 Forban (Le) NB
Feta  NB
 Poivroux (Le) NB
 Petit lardé (Le) NB
Cancre Cancre NB
Brigand Brigand NB
Gavroche Gavroche NB
Petit frais aux noix et sirop d'érable Petit frais aux noix et sirop d'érable NB
Petits caprice Petits caprice NB
Petit frais aux canneberges Petit frais aux canneberges NB
